In [26]:
from datasets import load_dataset
import pathlib
import shutil

In [27]:
# def clean_ds_name():
#     path = pathlib.Path("data/town/train/Minowacho")
#     for file in list(path.iterdir()):
#         stem = file.stem.replace(".", "")
#         new_file = file.with_stem(stem)
#         file.rename(new_file.as_posix())

In [28]:
ds = load_dataset("imagefolder", data_dir="data/town")

Resolving data files: 100%|██████████| 1016/1016 [00:00<00:00, 12654.60it/s]
Found cached dataset imagefolder (/Users/fumiyatanaka/.cache/huggingface/datasets/imagefolder/default-5f321aa3f7e3a7d9/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
100%|██████████| 1/1 [00:00<00:00, 274.07it/s]


In [29]:
ds["train"][10]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x640>,
 'label': 0}

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("geolocal/StreetCLIP")

def encode(example):
    # Adjust depending on your dataset's structure
    text = example["text"]
    return tokenizer(text, truncation=True, padding="max_length")


In [31]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, AutoProcessor

class CLIPDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.processor = AutoProcessor.from_pretrained("geolocal/StreetCLIP")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["text"]
        image = Image.open(item["image_path"]).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        inputs = self.processor(text, images=image, return_tensors="pt", padding="max_length", max_length=77)
        return inputs

# Transformations for the image
transform = transforms.Compose([
    transforms.Resize((336, 336)),
    transforms.ToTensor(),
    transforms.Normalize([0] * 3, [1] * 3)
])

In [32]:

data = []

folders = list(pathlib.Path("./data/town/train/").iterdir())

for folder in folders:
    folder_name = folder.name
    if folder_name == ".DS_Store":
        continue
    for file in list(folder.iterdir()):
        d = {
            "text": folder_name,
            "image_path": file
        }
        data.append(d)

# Initialize the dataset
dataset = CLIPDataset(data, transform)


In [33]:
from transformers import AutoModel, TrainingArguments, Trainer

# Load the model
model = AutoModel.from_pretrained("geolocal/StreetCLIP")

# Specify the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Create the Trainer and train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Train the model
trainer.train()


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`